In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'/ECE_285_proj/
%cd DeepZip_Compression

ModuleNotFoundError: No module named 'google.colab'

In [1]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [75]:
import sys
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers import LSTM, Flatten, Conv1D, LocallyConnected1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from math import sqrt
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from matplotlib import pyplot
import keras
from sklearn.preprocessing import OneHotEncoder
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
import numpy as np
import argparse
import os
from keras.callbacks import CSVLogger
import keras.backend as K
np.set_printoptions(threshold=sys.maxsize)

## DATA PREP

In [76]:
import data_preparation
#Convert letters to integers
integer_encoded, params = data_preparation.process_data('input.txt')

15072434
{'A': 0, 'G': 1, 'C': 2, 'T': 3}
{0: 'A', 1: 'G', 2: 'C', 3: 'T'}
[[1]
 [2]
 [2]
 [3]
 [0]
 [0]
 [1]
 [2]
 [2]
 [3]]
GCCTAAGCCT


In [77]:
bs=128
time_steps=64
num_epochs=20

In [78]:
#Generate traning data and its lable 
X,Y_raw, Y = data_preparation.generate_single_output_data(integer_encoded[: bs * 350],bs, time_steps)


(44800, 1)
(44800,)
44672
(44672, 65)


C:\Users\quoca\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Trainning

In [79]:
def custom_loss(y_true, y_pred):
        return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)        

In [80]:
alphabet_size = Y.shape[1]
DZ_model = Sequential()
DZ_model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(Flatten())
DZ_model.add(Dense(64, activation='relu'))
DZ_model.add(Dense(alphabet_size, activation='softmax'))
optim = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
DZ_model.compile(loss=custom_loss, optimizer=optim)
early_stopping = EarlyStopping(monitor='loss', mode='min', min_delta=0.005, patience=3, verbose=1)
callbacks_list = [early_stopping]
DZ_model.fit(X, Y, epochs=num_epochs, batch_size=bs, verbose=1, shuffle=True, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 7168/44672 [===>..........................] - ETA: 27s - loss: 1.9436

KeyboardInterrupt: 

## Codec

In [83]:
class ArithmeticEncoder(object):
    def __init__(self, bitlen):
        self.bit_prec = bitlen   #bit precision 
        self.max_range = 1 << self.bit_prec  #max range based on bit precision 2^bit_prec
        self.mask = self.max_range - 1  #max range index starting at 0
        self.renorm= self.max_range >> 1  #renormalization threshold
        self.second_mask = self.max_range >> 1
        self.low = 0  #initial low
        self.high = self.mask  #initial high
        self.s = 0  

    def update(self, sym, c):
        low = self.low   
        high = self.high
        range = high - low + 1
        sym = int(sym)
        total = (c[-1])  #cumulative propabilities
        symlow = (c[sym])  
        symhigh = (c[sym+1] )
        

        newlow = low + symlow*range // total  #low in arithmetic integer
        newhigh = low + symhigh*range // total -1 #high in arithemtic integer
        self.low = int(newlow)
        self.high = int(newhigh)
        range = self.high - self.low 

        while((self.low ^ self.high) & self.renorm) == 0:
            self.low = (self.low << 1) 
            range = range << 1 | 1
            self.high = self.low + range
            self.s = self.s + 1          

   
    def write(self, c, sym):
        self.update(c,sym)
        return [self.low, self.s]
        
    def finish(self):
        self.output.write(1)





class ArithmeticDecoder(ArithmeticEncoder):
    def __init__(self,statesize,bitin,c):
        self.input = bitin
        self.bitstream = ([int(d) for d in str(self.input)])
        self.code = 0
        self.max_range = 1 << statesize
        self.renorm= self.max_range >> 1
        self.mask = self.max_range - 1
        self.stream = self.input[0:statesize]
        self.low = int(self.stream,2)
        self.t = statesize
        self.thresh = []
        self.total = (c[-1])
        self.cum = [c[i]/self.total for i in range(len(c))]
        self.thresh = [round((self.cum[i])*self.mask) for i in range(len(self.cum))]
        
    def decode(self, c):
        c = [c[i]/self.total for i in range(len(c))]
        for i in range(len(self.thresh)-1):
            if((self.low < self.thresh[i+1]) & (self.low >= self.thresh[i])):
                sym = i
                rangenew = int(self.thresh[i+1] - self.thresh[i])
                while (rangenew < self.renorm):
                    rangenew = rangenew << 1 
                    self.low = self.low << 1
                    self.thresh[i] = int(self.thresh[i]) << 1
                    self.thresh[i+1] = rangenew + self.thresh[i]
                self.thresh = [(c[j]*(self.thresh[i+1]-self.thresh[i]))+self.thresh[i] for j in range(len(self.cum))]
                break
        return sym

## Encoding

In [ ]:
#Load model
DZ_model = load_model('my_model.h5', compile=False)

bitprecision = 64
l = int(X.shape[0] / bs) * bs 

#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
c = np.zeros(alphabet_size+1,  dtype = np.int64)
c[1:] = np.cumsum(prob*10000000 + 1)        
bitstream = []
enc = ArithmeticEncoder(bitprecision)

num_iters = int((len(X)+time_steps)/bs)
ind = np.array(range(bs))*num_iters
count =0 


#Encoding
count = 0
new_l = []
for i in (range(bs)):
    for j in range(min(time_steps, num_iters)):
        if (count%16 == 0 and count != 0):
            enc = None
            enc = ArithmeticEncoder(bitprecision)
            new_l.append(new)
            new = None
        new = enc.write(X[ind[i], j], c.tolist())
        count = count + 1
new_l.append(new)

count = 0
cumul_array = np.zeros(( round((num_iters - time_steps) * bs), 5), dtype = np.uint64)
new_l1 = []
enc1 = ArithmeticEncoder(bitprecision)
#convert low to bitstream, ensure proper number of bits 
cumul = np.zeros((bs, alphabet_size + 1), dtype = np.uint64)
t = 0
for k in (range(num_iters - time_steps)):
    prob = DZ_model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob*10000000 + 1, axis = 1)
    for i in range(bs):
        if (count%16 == 0 and count != 0):
            enc1 = None
            enc1 = ArithmeticEncoder(bitprecision)
            new_l1.append(new)
        cumul_array[count, :] = cumul[i,:]
        new = enc1.write(Y_raw[ind[i]], cumul[i,:].tolist()) 
        count = count + 1
        
    ind = ind + 1
new_l1.append(new)
new = None





# Decoding

In [66]:
#Decoding first 64 uniform bits
series_2d = np.zeros((bs,num_iters), dtype = np.uint8)
symdec = []
count = 1
for i in range(len(new_l)):
    count = count + 1
    new = new_l[i]
    low_final = new[0] #final low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b') 
    while len(bitstream) < (s_final+bitprecision):
        bitstream = '0' + bitstream
    #START DECODER
    dec = None
    dec = ArithmeticDecoder(bitprecision, bitstream, c.tolist())
    for i in range(16):
        symdec.append(dec.decode(c.tolist()))   



In [67]:
# NN decoding for the next num_iter - time_steps
symdec_a = np.asarray(symdec)
symdec_a = symdec_a.reshape(128, 64)
series_2d = np.zeros((bs,num_iters), dtype = np.uint8)
series_2d[:,0:64] = symdec_a
bitprecision = 64
l = int(X.shape[0] / bs) * bs 
print(l)
print((X.shape[0])-time_steps)

#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
c = np.zeros(alphabet_size+1,  dtype = np.int64)
c[1:] = np.cumsum(prob*10000000 + 1)        
bitstream = []
enc = ArithmeticEncoder(bitprecision)

n_sym = []
count = 0
t = 0
cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)
for j in (range(num_iters - time_steps)):
    prob = DZ_model.predict(series_2d[:,j:j+time_steps], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob*10000000 + 1, axis = 1)
    for i in range(bs):
        if (count%16 == 0):
            new = new_l1[t]
            low_final = new[0] #final low value 
            s_final = new[1]  #count of renormalizations
            bitstream = format(int(low_final), 'b') 
            t = t + 1
            while len(bitstream) < (s_final+bitprecision):
                bitstream = '0' + bitstream
            c = cumul[i,:]   
        dec = ArithmeticDecoder(bitprecision, bitstream, c.tolist())
        c = cumul[i,:]
        n_sym.append(dec.decode(c.tolist()))
        series_2d[i,j+time_steps] = n_sym[-1]
        count = count + 1
        
        


               

44672
44608


In [69]:
 a= series_2d.reshape(-1)

In [74]:
a[64:350]

array([0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2,
       2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0,
       1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3,
       0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2,
       2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0,
       1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3,
       0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2,
       2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0,
       1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3,
       0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2,
       2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0,
       1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3,
       0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2],
      dtype=uint8)

In [73]:
Y_raw[0:286].T

array([[0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2,
        2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0,
        1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3,
        0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2,
        2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0,
        1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3,
        0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2,
        2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0,
        1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3,
        0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2,
        2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0,
        1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3,
        0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2, 2, 3, 0, 0, 1, 2]])